In [9]:
import pandas as pd
from collections import Counter
import time

In [10]:
class SolvedException(Exception):
    "Raised when the input value is right and the game is done"
    pass

class TriesUsedException(Exception):
    "Raised when the max tries are reached and the game is done"
    pass

In [11]:
class GermanWords:
    
    def set_dictionary_file(self, filename):
        self.filename = filename 
        
    def read_file(self):
        self.lines = ''
        with open (self.filename, 'r', encoding='latin1') as f:
            self.lines = f.readlines()
            
    def filter_to_char_len_words(self, charlen):
        self.filtered_lines = [line.strip().lower() for line in self.lines if len(line.strip()) == charlen]
            
    def remove_special_char_words(self):
        special_chars = set("äöüßéèëáàóòêâôûúùã¼")  # Add any other special characters to this set
        self.filtered_lines = [line for line in self.filtered_lines if not any(char in special_chars for char in line)]

    def get_filtered_list(self, dic_file, word_length):
        self.set_dictionary_file(dic_file)
        self.read_file()
        self.filter_to_char_len_words(word_length)
        self.remove_special_char_words()
        return self.filtered_lines

In [12]:
class Wordle:
    
    def set_word_to_solve(self, word):
        self.word_to_solve = word.lower()
        self.reset_words_used()
        self.reset_number_tries()
        self.calc_count_of_chars_solved_word()
        
    def set_wordlist(self, wordlist):
        self.wordlist = wordlist
        
    def is_word_already_used(self, word):
        return word in self.words_used
    
    def is_word_in_wordlist(self, word):
        return word in self.wordlist
    
    def higher_number_tries(self):
        self.num_tries += 1
        
    def reset_number_tries(self):
        self.num_tries = 0
        
    def reset_words_used(self):
        self.words_used = []
        
    def set_maximum_number_tries(self, num_tries_max):
        self.max_tries = num_tries_max
        
    def set_word_input(self, word):
        if self.is_word_already_used(word):
            print(self.words_used)
            raise Exception("Word already used") 
        elif not self.is_word_in_wordlist(word):
            raise Exception("Word not in list")
        else: 
            self.word_input = word.lower()
            self.words_used.append(self.word_input)
    
    def is_maxium_tries(self):
        return self.num_tries >= self.max_tries
    
    def get_word_input(self):
        return self.word_input
        
    def get_input_with_space(self):
        return " ".join(list(self.word_input))
    
    def calc_count_of_chars_solved_word(self):
        chars = {}
        for char in self.word_to_solve:
            chars[char] = chars.get(char, 0) + 1
        self.chars_count_solve = chars
        
    def get_chars_count_solve(self):
        return self.chars_count_solve
    
    def clean_results_duplicate_chars(self):
        for key, value in self.chars_count_solve.items():
            char_left = value
            num_right = len(self.result_df[(self.result_df.Character == key) & (self.result_df.Rating == 2)])
            char_left -= num_right
            self.result_df.loc[(self.result_df.Character == key) & (self.result_df.Rating == 1), 'Rating'] = 0
            self.result_df.loc[self.result_df[(self.result_df.Character == key) & (self.result_df.Rating == 0)].index[:char_left], 'Rating'] = 1
                
    def check_word(self):
        result = ''
        char_checked = {}
        for i in range(0, word_length):
            char_input = self.word_input[i]
            char_to_solve = self.word_to_solve[i]
            if char_input == char_to_solve:
                result += '2'
                char_checked[char_input] = char_checked.get(char_input, 0) + 1
            elif (char_input in self.word_to_solve):
                result += '1'
                char_checked[char_input] = char_checked.get(char_input, 0) + 1
            else:
                result += '0'
        self.result = result
        if self.get_result_right():
            raise SolvedException("Great, you solved it!")
        self.higher_number_tries()
        if self.is_maxium_tries():
            raise TriesUsedException("Sorry, not solved!")
        return self.result
    
    def get_result(self):
        print(self.get_input_with_space())
        print(self.check_word())
        
    def get_result_as_table(self):
        wordle_result = self.check_word()
        result = pd.DataFrame(columns = ['Character', 'Rating'])
        for i in range(0, word_length):
            new_row = pd.DataFrame({'Character': self.word_input[i], 'Rating' : int(wordle_result[i])}, index=[-1])
            result = pd.concat([result.loc[:], new_row]).reset_index(drop=True)
        self.result_df = result
        self.clean_results_duplicate_chars()
        return self.result_df
    
    def get_result_right(self):
        result_to_be = ''
        for i in range(0, len(self.word_to_solve)):
            result_to_be += '2'
        return self.result == result_to_be
    
    def setup(self, dic, num_tries, word):
        wordle.set_wordlist(lines)
        wordle.set_maximum_number_tries(num_tries)
        wordle.set_word_to_solve(word)
        
    def check_word_result_table(self, word):
        self.set_word_input(word)
        return self.get_result_as_table()

In [24]:
class Solver:
    
    def __init__(self):
        self.used_words = []
    
    def set_results(self, results):
        self.results_df = results
        
    def set_dictionary(self, lines):
        self.filtered_lines = lines
        print('Possible Words: '+ self.get_stats())
        
    def get_filtered_lines(self):
        return self.filtered_lines
    
    def remove_duplicates(self):
        self.filtered_lines = list(set(self.filtered_lines))
        
    def filter_results_by_right_chars_right_place(self):
        chars_in_place = self.results_df[self.results_df.Rating == 2].Character.to_dict()
        lines_filtered = []
        if len(chars_in_place) > 0:
            for line in self.filtered_lines:
                for i, place in enumerate(chars_in_place):
                    if place >= len(line) or chars_in_place[place].lower() != line[place].lower():
                        break
                else:
                    lines_filtered.append(line)  
        else:
            lines_filtered = self.filtered_lines
        self.filtered_lines = lines_filtered
        self.remove_duplicates()
        
    def filter_results_by_right_chars_wrong_place(self):
        chars_right = self.results_df[self.results_df.Rating == 1].Character.to_dict()
        lines_filtered = []
        if len(chars_right) > 0:
            for line in self.filtered_lines:
                if set(chars_right.values()).intersection(set(line)):
                    lines_filtered.append(line)
                    continue
                all_not_in_place = False
                for i in chars_right:
                    if chars_right[i] != line[i]:
                        all_not_in_place = True
                    else:
                        break
                if(all_not_in_place):
                    lines_filtered.append(line) 
        else:
            lines_filtered = self.filtered_lines
        self.filtered_lines = lines_filtered
        self.remove_duplicates()
        
    def filter_results_by_false_chars(self):
        chars_not_used = (
            set(self.results_df[self.results_df.Rating == 0].Character) 
            - set(self.results_df[self.results_df.Rating == 2].Character) 
            - set(self.results_df[self.results_df.Rating == 1].Character)
        )
        if chars_not_used:
            self.filtered_lines = [line for line in self.filtered_lines if not chars_not_used.intersection(line)]
            
    def filter_results_by_limited_chars(self): 
        chars_rating_0 = set(self.results_df[self.results_df.Rating == 0].Character)
        chars_rating_1_2 = set(self.results_df[self.results_df.Rating.isin([1, 2])].Character)
        chars_duplicates = chars_rating_0 & chars_rating_1_2
        result_df_rating_1_2 = self.results_df[self.results_df.Rating.isin([1, 2])]
        lines_filtered = [
            line for line in self.filtered_lines 
            if all(line.count(char) <= len(result_df_rating_1_2[result_df_rating_1_2.Character == char]) 
                   for char in chars_duplicates)
        ]
        self.filtered_lines = lines_filtered        
        
    def filter_results(self):
        self.filter_results_by_right_chars_right_place()
        self.filter_results_by_right_chars_wrong_place()
        self.filter_results_by_false_chars()
        self.filter_results_by_limited_chars()
        
    def calculate_next_word(self, results):
        self.set_results(results)
        self.filter_results()
    
    def calculate_most_common_words(self):
        words = solver.get_filtered_lines()
        char_counts = Counter(''.join(words))
        total_chars = sum(char_counts.values())
        normalized_freqs = {char: round(freq / total_chars, 3) for char, freq in char_counts.items()}
            
        #words_percentage = {}
        #for word in words:
        #    char_percentage = 0
        #    char_used = []
        #    for char in word:
        #        count = char_used.count(char) + 1
        #        if(count > 1):
        #            count *= 5
        #        char_percentage += (normalized_freqs[char] * (1/count))
        #        char_used.append(char)
        #    words_percentage[word] = char_percentage
                
        words_percentage = {word: round(sum(normalized_freqs[char] for char in set(word)), 3) for word in words}
        words_percentage = dict(sorted(words_percentage.items(), key=lambda item: item[1], reverse=True))
        self.words_percentage = words_percentage
        
    def retrieve_most_common_word(self):
        words = list(self.words_percentage)
        index = 0
        word = words[index]
        while word in self.used_words:
            index += 1
            word = words[index]
        return word
        
    def get_stats(self):
        return str(len(self.filtered_lines))
    
    def get_most_common_word(self):
        self.calculate_most_common_words()
        return self.retrieve_most_common_word()
    
    def get_next_word(self, results):
        self.calculate_next_word(results)
        word = self.get_most_common_word()
        self.used_words.append(word)
        return word
    
    def solve_wordle(self, lines, wordle):
        self.used_words = []
        self.set_dictionary(lines)
        counter = 1
        df_result = pd.DataFrame(columns = ['Character', 'Rating'])
        solved = False
        first_word_used = False
        while True:
            word = self.get_next_word(df_result)
            print('Try ' + str(counter) + ' - ' + word)
            try:
                df_result = wordle.check_word_result_table(word)
            except SolvedException:
                print('Solved')
                solved = True
                break
            except TriesUsedException:
                print('Not solved')
                break
            print('Words left: '+ solver.get_stats())
            counter +=1
        return (counter, solved)

In [30]:
### Settings ###
num_tries = 6
word_length = 5
word_to_solve = 'Atome'

#Setup Dictionary
german = GermanWords()
lines = german.get_filtered_list('wordlist_wordle_at.txt', word_length)

#Setup Wordle 
wordle = Wordle()
wordle.setup(lines, num_tries, word_to_solve)

#Setup Solver
solver = Solver()
counter, solved = solver.solve_wordle(lines, wordle)

Possible Words: 8258
Try 1 - stare
Words left: 8233
Try 2 - atome
Solved


In [ ]:
#TODO
#Bessers Startwort
#UI 

In [33]:
### Testing ###

### Settings ###
num_tries = 6
word_length = 5

german = GermanWords()
wordle = Wordle()
solver = Solver()
lines = german.get_filtered_list('wordlist_wordle_at.txt', word_length)

df_word_results = pd.DataFrame(columns = ['Word', 'Solved', 'Counter', 'Time'])

for word_to_solve in lines:
    wordle.setup(lines, num_tries, word_to_solve)
    ts_start = time.time()
    counter, solved = solver.solve_wordle(lines, wordle)
    ts_end = time.time()
    ts_diff = round(ts_end - ts_start, 4)
    new_row = pd.DataFrame(
        {
            'Word': word_to_solve, 
            'Solved' : solved, 
            'Counter' : counter,
            'Time' : ts_diff
        }, 
        index=[-1]
    )
    df_word_results = pd.concat([df_word_results.loc[:], new_row]).reset_index(drop=True)
    
df_word_results

Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - luchs
Words left: 42
Try 3 - disco
Words left: 6
Try 4 - fisch
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - ritus
Words left: 146
Try 3 - troll
Words left: 20
Try 4 - trotz
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - reuig
Words left: 337
Try 3 - loipe
Words left: 191
Try 4 - hirse
Words left: 36
Try 5 - birke
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - rufen
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - loben
Words left: 81
Try 3 - nobel
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - luchs
Words left: 42
Try 3 - modus
Words left: 3
Try 4 - fokus
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - reuig
Words left: 337
Try 3 - rudel
Words left: 177
Try 4 - buero
Words left: 92
Try 5 - kreuz
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - malen
Words left: 117
Try 3 - sahne
Words left: 6

Words left: 5
Try 6 - prosa
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - moral
Words left: 177
Try 3 - vorab
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - liter
Words left: 88
Try 3 - moder
Words left: 55
Try 4 - trueb
Words left: 25
Try 5 - trieb
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - rudel
Words left: 70
Try 3 - imker
Words left: 22
Try 4 - woher
Words left: 8
Try 5 - speer
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - selig
Words left: 198
Try 3 - kuehl
Words left: 64
Try 4 - loewe
Words left: 3
Try 5 - eleve
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - stiel
Words left: 306
Try 3 - stufe
Words left: 9
Try 4 - stube
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - zutun
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - salbe
Words left: 64
Try 3 - maske
Words left: 21
Try 4 - kasse
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1

Try 1 - raten
Words left: 1147
Try 2 - humor
Words left: 75
Try 3 - lyrik
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - stiel
Words left: 306
Try 3 - demut
Words left: 63
Try 4 - tempo
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - stich
Words left: 84
Try 3 - somit
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - malen
Words left: 117
Try 3 - sahne
Words left: 61
Try 4 - wange
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - linse
Words left: 334
Try 3 - wenig
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - salon
Words left: 6
Try 3 - dahin
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - bogen
Words left: 44
Try 3 - besen
Words left: 24
Try 4 - heben
Words left: 9
Try 5 - ueben
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - bogen
Words left: 44
Try 3 - hegen
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - schal
Words left: 101
Try 3 

Words left: 334
Try 3 - sonde
Words left: 54
Try 4 - suche
Words left: 7
Try 5 - senke
Words left: 3
Try 6 - szene
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - loben
Words left: 58
Try 3 - wohin
Words left: 4
Try 4 - foehn
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - rasen
Words left: 169
Try 3 - narbe
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - selig
Words left: 198
Try 3 - kelch
Words left: 9
Try 4 - beleg
Words left: 6
Try 5 - welpe
Words left: 5
Try 6 - felge
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - ruhig
Words left: 3
Try 3 - rumpf
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - linse
Words left: 334
Try 3 - linde
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - minus
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - reuig
Words left: 31
Try 3 - recht
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - reuig
Words left: 3

Words left: 35
Try 3 - runde
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - linse
Words left: 334
Try 3 - linde
Words left: 16
Try 4 - linke
Words left: 14
Try 5 - loipe
Words left: 12
Try 6 - leine
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - bogen
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - ruhig
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - stiel
Words left: 306
Try 3 - sucht
Words left: 22
Try 4 - spott
Words left: 5
Try 5 - sekte
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - bogen
Words left: 44
Try 3 - wogen
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - schal
Words left: 101
Try 3 - umbau
Words left: 19
Try 4 - kampf
Words left: 10
Try 5 - dogma
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - loben
Words left: 81
Try 3 - insel
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - selig
Words left: 198
Try 3 - oelig
Words l

...

Words left: 64
Try 3 - macke
Words left: 16
Try 4 - bache
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - salto
Words left: 183
Try 3 - stahl
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - moral
Words left: 254
Try 3 - urban
Words left: 85
Try 4 - frack
Words left: 10
Try 5 - drang
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - selig
Words left: 198
Try 3 - hocke
Words left: 25
Try 4 - kuppe
Words left: 10
Try 5 - zweck
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - somit
Words left: 122
Try 3 - sonst
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - rufen
Words left: 5
Try 3 - roden
Words left: 3
Try 4 - reden
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - rudel
Words left: 70
Try 3 - imker
Words left: 22
Try 4 - woher
Words left: 8
Try 5 - foyer
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - schal
Words left: 101
Try 3 - klima
Words left: 61
Try 4 - 

Words left: 9
Try 4 - stube
Words left: 2
Try 5 - stute
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - salbe
Words left: 64
Try 3 - macke
Words left: 12
Try 4 - waffe
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - stich
Words left: 84
Try 3 - stopp
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - malen
Words left: 117
Try 3 - nabel
Words left: 93
Try 4 - sahne
Words left: 73
Try 5 - lange
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - reuig
Words left: 337
Try 3 - lehre
Words left: 49
Try 4 - bevor
Words left: 4
Try 5 - serum
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - selig
Words left: 198
Try 3 - folge
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - linse
Words left: 334
Try 3 - zunge
Words left: 83
Try 4 - bohne
Words left: 39
Try 5 - zweck
Words left: 8
Try 6 - enzym
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - reuig
Words left: 337
Try 3 - r

Try 1 - raten
Words left: 1147
Try 2 - linse
Words left: 334
Try 3 - kunde
Words left: 36
Try 4 - wunde
Words left: 10
Try 5 - pumpe
Words left: 9
Try 6 - zunge
Not solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - luchs
Words left: 42
Try 3 - dumpf
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - ruine
Words left: 507
Try 3 - linse
Words left: 155
Try 4 - birne
Words left: 21
Try 5 - rinde
Words left: 17
Try 6 - pinie
Not solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - malen
Words left: 117
Try 3 - nabel
Words left: 93
Try 4 - sahne
Words left: 73
Try 5 - laune
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - bogen
Words left: 44
Try 3 - eisen
Words left: 16
Try 4 - ulken
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - selig
Words left: 198
Try 3 - suche
Words left: 10
Try 4 - socke
Solved
Possible Words: 1147
Try 1 - raten
Words left: 1147
Try 2 - minus
Words left: 107
Try 3 - fluor
Words left:

Words left: 306
Try 3 - suite
Solved


,Word,Solved,Counter,Time
0,fisch,True,4,0.0292
1,trotz,True,4,0.0242
2,birke,True,5,0.0329
3,rufen,True,2,0.0103
4,nobel,True,3,0.0173
...,...,...,...,...
1142,macho,True,3,0.0166
1143,fahrt,True,3,0.0183
1144,taler,True,2,0.012
1145,werft,True,5,0.0357


In [34]:
## Results with Dict with two chars##
num_solved = len(df_word_results[df_word_results.Solved == True])
percentage_solved = round(num_solved /len(df_word_results) * 100, 2)
mean_time = round(df_word_results[df_word_results.Solved == True].Time.mean(), 4)
mean_counter = round(df_word_results[df_word_results.Solved == True].Counter.mean(), 2)

print('Solved: ' + str(num_solved) + ' of ' + str(len(df_word_results)) + ' (' + str(percentage_solved) + '%)')
print('Mean time per word: ' + str(mean_time) + 's')
print('Mean tries until solved: ' + str(mean_counter) + ' tries')

Solved: 1099 of 1147 (95.82%)
Mean time per word: 0.0251s
Mean tries until solved: 4.11 tries


In [17]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    
    
### Settings ###
num_tries = 6
word_length = 5
word_to_solve = 'atome'

#Setup Dictionary
german = GermanWords()
lines = german.get_filtered_list('wordlist-german.txt', word_length)

#Setup Wordle 
wordle = Wordle()
wordle.setup(lines, num_tries, word_to_solve)

results = {
    0 : '',
    1 : '',
    2 : '',
    3 : '',
    4 : '',
    5 : ''
}

df_result = wordle.check_word_result_table('wohne')

for num_try in results:
    if results[num_try] == '':
        results[num_try] = df_result
    break

for i in range(0, num_tries):
    print(word_length * ' ---    ')
    
    df_word = results[i]
    string = ''
    for ix in range(0, word_length):
        if len(df_word) > 0:
            row = df_word.iloc[ix]
            if row.Rating == 0:
                color = bcolors.FAIL
            elif row.Rating == 1:
                color = bcolors.WARNING
            else:
                color = bcolors.OKGREEN
            string += '| ' + color + row.Character.upper()  + bcolors.ENDC + ' |   '
        else:
            string += '|   |   '
    print(string)
    print(word_length * ' ---    ')
    
#TODO Keyboard

 ---     ---     ---     ---     ---    
| W |   | O |   | H |   | N |   | E |   
 ---     ---     ---     ---     ---    
 ---     ---     ---     ---     ---    
|   |   |   |   |   |   |   |   |   |   
 ---     ---     ---     ---     ---    
 ---     ---     ---     ---     ---    
|   |   |   |   |   |   |   |   |   |   
 ---     ---     ---     ---     ---    
 ---     ---     ---     ---     ---    
|   |   |   |   |   |   |   |   |   |   
 ---     ---     ---     ---     ---    
 ---     ---     ---     ---     ---    
|   |   |   |   |   |   |   |   |   |   
 ---     ---     ---     ---     ---    
 ---     ---     ---     ---     ---    
|   |   |   |   |   |   |   |   |   |   
 ---     ---     ---     ---     ---    
